## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits.

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because we will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [2]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', as_supervised = True, split = 'train')

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image) 

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', as_supervised = True, split = 'test')

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image) 

# batch the dataset
test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete6EZGZV/cifar10-train.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incomplete6EZGZV/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


## Build the Model

Create the autoencoder model. We will downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Input images will have the shape `(32, 32, 3)`.

In [3]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D

# use the Sequential API (you can remove if you want to use the Functional API)
#model = Sequential()

# use `model.add()` to add layers (if using the Sequential API)


# def encoder(inputs):
#   '''Defines the encoder with two Conv2D and max pooling layers.'''
#   conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(inputs)
#   max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv_1)

#   conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(max_pool_1)
#   max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv_2)

#   return max_pool_2




# def bottle_neck(inputs):
#   '''Defines the bottleneck.'''
#   bottle_neck = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same')(inputs)
#   encoder_visualization = tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')(bottle_neck)

#   return bottle_neck, encoder_visualization




# def decoder(inputs):
#   '''Defines the decoder path to upsample back to the original image size.'''
#   conv_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(inputs)
#   up_sample_1 = tf.keras.layers.UpSampling2D(size=(2,2))(conv_1)

#   conv_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(up_sample_1)
#   up_sample_2 = tf.keras.layers.UpSampling2D(size=(2,2))(conv_2)

#   conv_3 = tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')(up_sample_2)

#   return conv_3


# def convolutional_auto_encoder():
#   '''Builds the entire autoencoder model.'''
#   inputs = tf.keras.layers.Input(shape=(32, 32, 3,))
#   encoder_output = encoder(inputs)
#   bottleneck_output, encoder_visualization = bottle_neck(encoder_output)
#   decoder_output = decoder(bottleneck_output)
  
#   model = tf.keras.Model(inputs =inputs, outputs=decoder_output)
#   encoder_model = tf.keras.Model(inputs=inputs, outputs=encoder_visualization)
#   return model, encoder_model



# model , enoder_model = convolutional_auto_encoder() 



model = Sequential()

model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.BatchNormalization())     # 32x32x32
model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'))      # 16x16x32
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      # 16x16x32
model.add(tf.keras.layers.BatchNormalization())     # 16x16x32
model.add(UpSampling2D())
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      # 32x32x32
model.add(tf.keras.layers.BatchNormalization()) 
model.add(Conv2D(3,  kernel_size=1, strides=1, padding='same', activation='sigmoid'))   # 32x32x3

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9

## Configure training parameters

In [4]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training

In [5]:
# parameters
train_steps = 50000 // BATCH_SIZE 
val_steps = 10000 // BATCH_SIZE

model.fit(train_dataset, validation_data=test_dataset, epochs=7)

Epoch 1/7
391/391 [==============================] - 52s 54ms/step - loss: 0.0046 - accuracy: 0.7194 - val_loss: 0.0073 - val_accuracy: 0.7149
Epoch 2/7
391/391 [==============================] - 15s 38ms/step - loss: 0.0012 - accuracy: 0.8027 - val_loss: 0.0010 - val_accuracy: 0.8031
Epoch 3/7
391/391 [==============================] - 15s 38ms/step - loss: 8.1521e-04 - accuracy: 0.8300 - val_loss: 7.2263e-04 - val_accuracy: 0.8258
Epoch 4/7
391/391 [==============================] - 15s 38ms/step - loss: 6.8482e-04 - accuracy: 0.8439 - val_loss: 6.0633e-04 - val_accuracy: 0.8268
Epoch 5/7
391/391 [==============================] - 15s 38ms/step - loss: 6.0406e-04 - accuracy: 0.8535 - val_loss: 6.2752e-04 - val_accuracy: 0.8242
Epoch 6/7
391/391 [==============================] - 15s 38ms/step - loss: 5.5568e-04 - accuracy: 0.8674 - val_loss: 4.5476e-04 - val_accuracy: 0.8582
Epoch 7/7
391/391 [==============================] - 15s 39ms/step - loss: 5.1160e-04 - accuracy: 0.8718 - val

## Model evaluation

In [6]:
result = model.evaluate(test_dataset, steps=10)

10/10 [==============================] - 0s 21ms/step - loss: 4.6860e-04 - accuracy: 0.8445


<img src="https://drive.google.com/uc?export=view&id=12Fy-guiP-3tTPfc9IV2nHhqLvs7LwRo6" width="75%" height="75%"/>

## Save model

In [7]:
model.save('mymodel.h5')